# https://www.kaggle.com/c/digit-recognizer/

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

%load_ext blackcellmagic

# dataset

In [2]:
batch_size = 200
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()]),
    ),
    batch_size=batch_size,
    shuffle=True,
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data", train=False, transform=transforms.Compose([transforms.ToTensor()])
    ),
    batch_size=batch_size,
    shuffle=True,
)

## Load data from kaggle

In [3]:
import pandas as pd
import numpy as np

train_csv = pd.read_csv('train.csv')
x = train_csv.values[:,1:].astype(np.float32)
y = train_csv.values[:,0]

x = x.reshape([-1, 28, 28, 1])

In [4]:
print(train_csv.shape)
print(len(train_loader.dataset))

from torch.utils.data import Dataset, DataLoader

class MnistDataset(Dataset):
    
    def __init__(self, x, y=None, transform=None):
        self.data = x
        self.labels = y
        self.transform = transform
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        if self.labels is not None:
            return self.transform(self.data[idx]), self.labels[idx]
        else:
            return self.transform(self.data[idx])


csv_dataloader = DataLoader(MnistDataset(x, y, transform=transforms.Compose([transforms.ToTensor()])), batch_size=batch_size, shuffle=True)

(42000, 785)
60000


# model

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc1 = nn.Linear(7 * 7 * 32, 128)
        self.dropout = nn.Dropout()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

m = CNN()
o = m(torch.rand(1, 1, 28, 28))
print(o.size())

torch.Size([1, 10])


In [6]:
class FiveLayerCNN(nn.Module):
    
    def __init__(self):
        super(FiveLayerCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.fc1 = nn.Linear(256*5*5, 128)
        self.dropout = nn.Dropout()
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
#         print(out.size())
        out = out.reshape(out.size(0), -1)
#         print(out.size())
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out
        
    
m = FiveLayerCNN()
o = m(torch.rand(1, 1, 28, 28))
print(o.size())

torch.Size([1, 10])


# train

In [11]:
model = CNN().cuda()
model.load_state_dict(torch.load("mnist_cnn.pt"))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.000001)
epochs = 10

for epoch in range(1, epochs + 1):
    print("Epoch [{}/{}]".format(epoch, epochs))

    # train
    model.train()

    train_loss = 0
    train_correct = 0

    for data1, data2 in zip(train_loader, csv_dataloader):
        inputs = torch.cat((data1[0].cuda(), data2[0].cuda()), 0)
        target = torch.cat((data1[1].cuda(), data2[1].cuda()), 0)
        optimizer.zero_grad()
        outputs = model(inputs)
        pred = outputs.max(1, keepdim=True)[
            1
        ]  # get the index of the max log-probability
        train_correct += pred.eq(target.view_as(pred)).sum().item()

        loss = criterion(outputs, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(
        "Train set: Average loss: {:.6f}, Accuracy: {}/{} ({:.3f}%)".format(
            train_loss / (len(train_loader.dataset) + len(csv_dataloader.dataset)),
            train_correct,
            len(train_loader.dataset) + len(csv_dataloader.dataset),
            100.0
            * train_correct
            / (len(train_loader.dataset) + len(csv_dataloader.dataset)),
        )
    )

    # evaluate
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.max(1, keepdim=True)[
                1
            ]  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "Test set: Average loss: {:.6f}, Accuracy: {}/{} ({:.3f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

torch.save(model.state_dict(), "mnist_cnn.pt")

Epoch [1/10]
Train set: Average loss: 0.000243, Accuracy: 81235/102000 (79.642%)
Test set: Average loss: 0.000924, Accuracy: 9456/10000 (94.560%)

Epoch [2/10]
Train set: Average loss: 0.000246, Accuracy: 81180/102000 (79.588%)
Test set: Average loss: 0.000922, Accuracy: 9456/10000 (94.560%)

Epoch [3/10]
Train set: Average loss: 0.000244, Accuracy: 81236/102000 (79.643%)
Test set: Average loss: 0.000919, Accuracy: 9460/10000 (94.600%)

Epoch [4/10]
Train set: Average loss: 0.000242, Accuracy: 81229/102000 (79.636%)
Test set: Average loss: 0.000918, Accuracy: 9459/10000 (94.590%)

Epoch [5/10]
Train set: Average loss: 0.000245, Accuracy: 81194/102000 (79.602%)
Test set: Average loss: 0.000917, Accuracy: 9464/10000 (94.640%)

Epoch [6/10]
Train set: Average loss: 0.000246, Accuracy: 81161/102000 (79.570%)
Test set: Average loss: 0.000917, Accuracy: 9466/10000 (94.660%)

Epoch [7/10]
Train set: Average loss: 0.000245, Accuracy: 81176/102000 (79.584%)
Test set: Average loss: 0.000916, Acc

In [8]:
!ls

data			mnist.ipynb	       submission.ipynb
mnist_cnn-bestScore.pt	sample_submission.csv  test.csv
mnist_cnn.pt		submission.csv	       train.csv
